### Começaremos agora a implementação dos modelos de regresão e classificação

In [18]:
!git clone https://github.com/Lucca0404/RegressaoLinear.git

fatal: destination path 'RegressaoLinear' already exists and is not an empty directory.


In [19]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys
!{sys.executable} -m pip install statsmodels
!{sys.executable} -m pip install tabulate
!{sys.executable} -m pip install --upgrade scikit-learn

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, root_mean_squared_error
import matplotlib.pyplot as plt

In [20]:
!{sys.executable} -m pip install pycaret

  Using cached pycaret-3.3.2-py3-none-any.whl.metadata (17 kB)
  Using cached numpy-1.26.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pandas-2.1.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (18 kB)
  Using cached scipy-1.11.4-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached joblib-1.3.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached pyod-2.0.6-py3-none-any.whl.metadata (46 kB)
  Using cached category_encoders-2.9.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached matplotlib-3.7.5-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.7 kB)
  Using cached scikit_plot-0.3.7-py3-none-any.whl.metadata (7.1 kB)
  Using cached kaleido-1.2.0-py3-none-any.whl.metadata (5.6 kB)
  Using cached schemdraw-0.15-py3-none-any.whl.metadata (2.2 kB)
  Using cached plotly_resampler-0.11.0-py3-none-any.whl.metadata (13 kB)
  Using cached sktime-0.26.0-py3-none-any.whl.met

---

##### 1. Regressão Linear Simples
**Hipótese H1:** Existe uma relação física direta entre a energia e o volume de uma música?
* **Premissa:** Músicas percebidas como "enérgicas" são masterizadas com volumes mais altos (Loudness War).
* **Variáveis:**
    * $X$ (Independente): `energia`
    * $y$ (Alvo): `volume` (dB)



In [21]:
# 1. Carregar Dataset
df = pd.read_csv('/content/RegressaoLinear/data/processed/dataset_limpo.csv')

print("=== H1: Regressão Linear Simples (Abordagem Estatística & Pipeline) ===")

# Definir X e y
X = df[['energia']]
y = df['volume']

# Divisão Treino/Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------------------------------------------------------
# PARTE A: STATSMODELS
# -----------------------------------------------------------------------------
print("\n--- Relatório Estatístico (Statsmodels - OLS) ---")

# O statsmodels precisa que adicionemos a constante (o intercepto/viés) manualmente
X_train_sm = sm.add_constant(X_train)

# Criamos o modelo OLS (Ordinary Least Squares)
modelo_sm = sm.OLS(y_train, X_train_sm).fit()

# O "Ouro": A tabela completa
print(modelo_sm.summary())


# -----------------------------------------------------------------------------
# PARTE B: PIPELINE DO SKLEARN (O jeito 'Engenharia de ML' de fazer)
# -----------------------------------------------------------------------------
print("\n--- Implementação via Pipeline (Sklearn) ---")

# Criamos um Pipeline que faz:
# 1. Escalonamento (StandardScaler) -> Deixa os dados na mesma escala (boa prática)
# 2. Modelo (LinearRegression)
pipeline_h1 = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

# Treinamos o pipeline inteiro de uma vez
pipeline_h1.fit(X_train, y_train)

# Previsão e Métricas
y_pred = pipeline_h1.predict(X_test)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = root_mean_squared_error(y_test, y_pred)

print(f"R² (Pipeline): {r2:.4f}")
print(f"MAE (Pipeline): {mae:.4f}")
print(f"RMSE (Pipeline): {rmse:.4f}")

=== H1: Regressão Linear Simples (Abordagem Estatística & Pipeline) ===

--- Relatório Estatístico (Statsmodels - OLS) ---
                            OLS Regression Results                            
Dep. Variable:                 volume   R-squared:                       0.585
Model:                            OLS   Adj. R-squared:                  0.585
Method:                 Least Squares   F-statistic:                 1.276e+05
Date:                Wed, 10 Dec 2025   Prob (F-statistic):               0.00
Time:                        22:09:15   Log-Likelihood:            -2.3124e+05
No. Observations:               90430   AIC:                         4.625e+05
Df Residuals:                   90428   BIC:                         4.625e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------

In [ ]:


# 1. Calcular métricas rápidas para colocar na legenda (Enfeite útil)
X = df[['energia']]
y = df['volume']
model = LinearRegression()
model.fit(X, y)
r2 = r2_score(y, model.predict(X))
coef = model.coef_[0]
intercept = model.intercept_

# 2. Configuração Estética do Gráfico
plt.figure(figsize=(12, 7))
sns.set_style("whitegrid") # Fundo branco com grades para facilitar leitura

# 3. O Gráfico Principal (Scatter + Linha)
# regplot do Seaborn já faz a regressão e o intervalo de confiança (sombra)
ax = sns.regplot(
    x='energia',
    y='volume',
    data=df,
    scatter_kws={'alpha': 0.1, 'color': '#555555', 's': 10}, # Pontos discretos e transparentes
    line_kws={'color': '#E63946', 'linewidth': 2, 'label': f'Linha de Regressão (R²={r2:.2f})'} # Linha Vermelha destaque
)

# 4. Adicionar a Equação Matemática no canto
texto_equacao = f'$Volume = {coef:.2f} \cdot Energia + ({intercept:.2f})$'
plt.text(0.05, 0.9, texto_equacao, transform=ax.transAxes,
         fontsize=14, fontweight='bold',
         bbox=dict(facecolor='white', alpha=0.9, edgecolor='gray', boxstyle='round,pad=0.5'))

# 5. Títulos e Eixos
plt.title('H1: Relação entre Energia e Volume (Loudness)', fontsize=16, pad=20)
plt.xlabel('Energia (Intensidade Perceptiva)', fontsize=12)
plt.ylabel('Volume (dB)', fontsize=12)
plt.legend(loc='lower right', frameon=True)

plt.tight_layout()
plt.show()

<>:25: SyntaxWarning: invalid escape sequence '\c'
<>:25: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipython-input-2114628952.py:25: SyntaxWarning: invalid escape sequence '\c'
  texto_equacao = f'$Volume = {coef:.2f} \cdot Energia + ({intercept:.2f})$'


### 📊 Conclusão da Hipótese H1: Energia vs. Volume

##### Veredito: Hipótese Confirmada ✅

A aplicação do modelo de **Regressão Linear Simples** validou a hipótese **H1**, demonstrando uma relação positiva entre a energia perceptiva de uma faixa e o seu volume (Loudness).

Os resultados sustentam-se em três pilares analíticos:

#### 1. Poder Explicativo Significativo ($R^2 \approx 0.58$)
O modelo atingiu um **Coeficiente de Determinação ($R^2$) de 0.58**.
* **Interpretação:** Aproximadamente **58% da variação do volume** em todo o dataset pode ser explicada exclusivamente pela variável `energia`. Para um modelo univariado (apenas uma variável de entrada), este é um resultado expressivo, indicando uma correlação estrutural forte.

#### 2. Precisão Auditiva Aceitável ($MAE \approx 2.2$ dB)
O **Erro Médio Absoluto (MAE)** foi de aproximadamente **2.2 dB**.
* **Interpretação:** No campo da psicoacústica, uma variação de 1 a 3 dB é considerada pequena para a percepção humana. O modelo consegue prever o volume de uma música com uma margem de erro que, na prática, é pouco perceptível ao ouvinte médio, tornando o modelo funcional para aplicações reais.

#### 3. Fundamentação na Engenharia de Som ("Loudness War")
A correlação positiva observada reflete práticas consolidadas da indústria fonográfica. Gêneros e faixas com alta energia (Pop, Rock, EDM) são frequentemente masterizados com maior compressão dinâmica para maximizar o volume percebido, enquanto faixas de baixa energia (Acústico, Clássico) preservam maior dinâmica e volumes médios menores.

---
**Conclusão:** O tratamento de outliers (remoção de faixas de silêncio e erros de medição) foi decisivo para revelar essa tendência. Podemos conjecturar que **a energia é um preditor confiável para o volume da música.**

---

##### 2. Regressão Linear Múltipla
**Hipótese H2:** A "dancabilidade" pode ser explicada por uma combinação de fatores técnicos?
* **Premissa:** Uma música não é dançante apenas pelo ritmo. A combinação de andamento (BPM), intensidade (Energia) e positividade (Valência) define o quão dançante ela é.
* **Variáveis:**
    * $X$ (Independentes): `tempo_bpm`, `energia`, `valencia`
    * $y$ (Alvo): `dancabilidade`

---

In [ ]:
# 1. Definir Variáveis
features = ['tempo_bpm', 'energia', 'valencia']
target = 'dancabilidade'

X = df[features]
y = df[target]

# 2. Divisão Treino/Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# -----------------------------------------------------------------------------
# PARTE A: ANÁLISE ESTATÍSTICA (STATSMODELS)
# -----------------------------------------------------------------------------
# Statsmodels exige adicionar constante manualmente para calcular o intercepto
X_train_sm = sm.add_constant(X_train)
modelo_sm = sm.OLS(y_train, X_train_sm).fit()

print("\n--- Summary do Modelo (Para Análise Acadêmica) ---")
print(modelo_sm.summary())

# -----------------------------------------------------------------------------
# PARTE B: PIPELINE DE PREDIÇÃO (SKLEARN)
# -----------------------------------------------------------------------------
# Pipeline garante que a normalização (StandardScaler) seja feita corretamente
pipeline_h2 = Pipeline([
    ('scaler', StandardScaler()),
    ('regressor', LinearRegression())
])

pipeline_h2.fit(X_train, y_train)
y_pred = pipeline_h2.predict(X_test)

# Métricas Finais
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"\n--- Resultados do Teste (Sklearn) ---")
print(f"R² Score: {r2:.4f}")
print(f"MAE: {mae:.4f}")

# Visualizar Importância das Features (Coeficientes)
coefs = pipeline_h2.named_steps['regressor'].coef_
feature_imp = pd.DataFrame({'Feature': features, 'Coeficiente (Impacto)': coefs})
print("\nO que define uma música dançante?")
print(feature_imp.sort_values(by='Coeficiente (Impacto)', ascending=False))

# Features da H2: (BPM, Energia, Valência)
X_features = df[['tempo_bpm', 'energia', 'valencia']].copy()

# Adiciona a constante (Intercepto) — Obrigatório para o cálculo do VIF
X_vif = sm.add_constant(X_features)

# Cria o DataFrame para armazenar os resultados
vif_data = pd.DataFrame()
vif_data["Feature"] = X_vif.columns

# Calcula o VIF para cada feature
vif_data["VIF"] = [variance_inflation_factor(X_vif.values, i)
                    for i in range(X_vif.shape[1])]
vif_data = vif_data.round(2)

print("\n--- Relatório VIF para Regressão Múltipla (H2) ---")
print(vif_data.to_markdown(index=False))

# -----------------------------------------------------------------------------
# PARTE C: GRÁFICO PREVISTO vs REAL
# -----------------------------------------------------------------------------
plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_test, y=y_pred, alpha=0.1, color='#6A0DAD', s=15)
# Linha de referência (Se acertasse tudo, estaria nessa linha)
plt.plot([0, 1], [0, 1], '--r', linewidth=2, label='Previsão Perfeita')

plt.xlabel('Dancabilidade Real')
plt.ylabel('Dancabilidade Prevista pelo Modelo')
plt.title(f'H2: Performance da Regressão Múltipla (R²={r2:.2f})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 🕺 Conclusão da Hipótese H2: O Que Faz uma Música Dançante?

#### Veredito: Hipótese Parcialmente Confirmada / Complexidade Revelada ⚠️

A aplicação da **Regressão Linear Múltipla** para prever a `dancabilidade` com base em `tempo_bpm`, `energia` e `valencia` revelou *insights* valiosos sobre a estrutura musical, embora tenha demonstrado que a dancabilidade não é puramente técnica.

Os resultados baseiam-se nos seguintes pilares:

##### 1. Poder Explicativo Moderado ($R^2 \approx 0.23$)
O modelo obteve um **$R^2$ de aproximadamente 0.23**.
* **Interpretação:** As variáveis escolhidas explicam cerca de **23% da variação da dancabilidade**. Isso indica que, embora BPM e Valência sejam importantes, a dancabilidade é um atributo subjetivo e complexo, dependendo fortemente de fatores não capturados aqui (como o padrão rítmico, o "groove" do baixo ou a síncope da bateria), e não apenas da velocidade ou intensidade.

##### 2. Valência: O Motor da Pista de Dança
A análise dos coeficientes padronizados revelou que a **Valência** (positividade) é o fator mais influente ($Coef \approx +0.08$).
* **Significado Prático:** Existe uma forte correlação entre músicas que soam "felizes/positivas" e sua adequação para dança. Músicas tristes, mesmo que rápidas, tendem a ser classificadas como menos dançantes pelo algoritmo do Spotify.

##### 3. O Paradoxo da Energia e do BPM
* **Energia ($Coef \approx 0.004$):** Surpreendentemente, a energia teve impacto quase nulo. Isso derruba o mito de que "mais barulho = mais dança". Um heavy metal (alta energia) pode ser difícil de dançar, enquanto um funk lento (média energia) pode ser muito dançante.
* **Tempo ($Coef \approx -0.019$):** O BPM apresentou uma leve correlação *negativa*. Isso sugere que aumentar a velocidade indefinidamente não torna a música mais dançante; pelo contrário, músicas muito rápidas podem se tornar caóticas, prejudicando a dancabilidade.

#### 4. O VIF revela que não há multicolinearidade
* como o VIF das váriaveis estão próximos de 1, isso significa que não há uma multicolinearidade significativa entre as variáveis

---
**Conclusão:** A dancabilidade é definida mais pela "vibe" (Valência) do que pela força bruta (Energia). A relação negativa com o BPM levanta a suspeita de que a relação ideal de tempo não é uma linha reta, mas uma curva, o que investigaremos na **Hipótese 3 (Polinomial)**.

---

##### 3. Regressão Polinomial
**Hipótese H3:** Existe uma "zona ideal" de BPM para dançar?
* **Premissa:** A relação entre BPM e Dancabilidade não é uma linha reta. Músicas muito lentas ou muito rápidas são menos dançantes; o ideal deve estar em uma faixa intermediária (curva).
* **Variáveis:**
    * $X$ (Independente): `tempo_bpm`
    * $y$ (Alvo): `dancabilidade`

In [ ]:
print("=== H3: REGRESSÃO POLINOMIAL (BPM -> Dancabilidade) ===")

# 1. Definir Variáveis
X = df[['tempo_bpm']]
y = df['dancabilidade']

# 2. Divisão Treino/Teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Criar o Pipeline com Transformação Polinomial
# O segredo: 'poly_features' transforma BPM em BPM e BPM^2
pipeline_h3 = Pipeline([
    ('scaler', StandardScaler()),                 # Garante que os números do polinômio não sejam muito grandes
    ('poly_features', PolynomialFeatures(degree=2)), # Cria as features X e X^2
    ('regressor', LinearRegression())             # Ajusta a melhor linha (curva) para essas novas features
])

# 4. Treinar e Prever
pipeline_h3.fit(X_train, y_train)
y_pred = pipeline_h3.predict(X_test)

# 5. Avaliação do Modelo
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"R² Score (Polinomial, Grau 2): {r2:.4f}")
print(f"MAE (Erro Médio Absoluto): {mae:.4f}")

# 6. VISUALIZAÇÃO DA CURVA
# Ordenamos o X de teste para que a linha de regressão seja plotada de forma suave
sort_idx = X_test['tempo_bpm'].argsort()
X_test_sorted = X_test.iloc[sort_idx]
y_pred_sorted = y_pred[sort_idx]

plt.figure(figsize=(10, 6))
# Plotamos os pontos reais
sns.scatterplot(x=X_test['tempo_bpm'], y=y_test, alpha=0.2, label='Dados Reais', color='gray')
# Plotamos a linha ajustada pelo modelo
plt.plot(X_test_sorted['tempo_bpm'], y_pred_sorted, color='darkorange', linewidth=3,
         label=f'Curva Polinomial (R²={r2:.2f})')

plt.title('H3: Regressão Polinomial (BPM -> Dancabilidade)', fontsize=16)
plt.xlabel('Tempo (BPM)', fontsize=12)
plt.ylabel('Dancabilidade', fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

### 📉 Conclusão da Hipótese H3: A Zona Ideal de Tempo (Polinomial)

#### Veredito: Forma da Relação Confirmada (Hipótese Validada) ✅

A aplicação da **Regressão Polinomial (Grau 2)** validou a hipótese de que a relação entre `tempo_bpm` e `dancabilidade` não é linear. O modelo capturou a curva esperada, provando a existência de um "ponto ideal" de andamento.

Os resultados demonstram:

##### 1. Confirmação da Não-Linearidade ($R^2 \approx 0.10$)
O modelo Polinomial de Grau 2 atingiu um **$R^2$ de aproximadamente 0.10** (10% de explicação).
* **Interpretação:** Embora o poder explicativo seja baixo (indicando que BPM é apenas um pequeno fator na dancabilidade)

##### 2. O Ponto de Ouro (U Invertido)
O gráfico visual confirma a forma de **parábola invertida** ("U invertido"):
* **Inferência:** A dancabilidade é baixa em BPMs muito lentos (60-80 BPM) e baixa novamente em BPMs muito rápidos (acima de 150 BPM). A curva atinge seu ponto máximo, o **ponto de saturação**, entre **120 e 130 BPM**—a faixa ideal para a maioria das músicas de dança modernas (como House, Disco e Pop-dance).

##### 3. Erro Consistente (MAE $\approx$ 0.13)
O Erro Médio Absoluto (MAE) de aproximadamente **0.13** (na escala 0 a 1) é baixo e similar ao da Regressão Múltipla (H2).
* **Significado Prático:** O modelo tem uma precisão razoável ao estimar a dancabilidade, e a adição do termo quadrático não introduziu instabilidade, indicando que a curva é matematicamente estável.

---
**Conclusão:** A Hipótese H3 é validada, pois conseguimos modelar e provar a **não-linearidade** da influência do tempo. A dancabilidade depende da Valência (H2) e de um ritmo que caia dentro da "zona ideal" (H3).

---

##### 4. Classificação (Naive Bayes)
**Hipótese H4:** Os gêneros musicais possuem "assinaturas acústicas" matemáticas?
* **Premissa:** É possível prever o gênero de uma música desconhecida analisando apenas suas características técnicas (sem saber o artista ou nome).
* **Variáveis:**
    * $X$ (Features): Todas as colunas numéricas de áudio (`acustica`, `instrumental`, `energia`, etc.)
    * $y$ (Target): `genero` (Categoria)

In [ ]:


# 1. CARREGAMENTO E LIMPEZA DE DADOS
df_clean = pd.read_csv('/content/RegressaoLinear/data/processed/dataset_limpo.csv')

# 2. Seleção de Features e Alvo
features = ['dancabilidade', 'energia', 'volume', 'falada',
            'acustica', 'instrumental', 'ao_vivo', 'valencia', 'tempo_bpm',
            'duracao_ms']
target = 'genero'
df_clf = df_clean[features + [target]].copy()

# 3. Filtrar Gêneros Raros
generos_validos = df_clf['genero'].value_counts()
generos_validos = generos_validos[generos_validos >= 1000].index
df_clf = df_clf[df_clf['genero'].isin(generos_validos)].reset_index(drop=True)

# 4. ENCODING do Alvo (y)
le = LabelEncoder()
y = le.fit_transform(df_clf[target])
X = df_clf[features]
classes_finais = le.classes_

# 5. DIVISÃO TREINO/TESTE (Corrigindo o problema de _validate_data com .values)
# Passamos X como array Numpy (.values) para contornar o erro de versão
X_train, X_test, y_train, y_test = train_test_split(
    X.values, y, test_size=0.2, random_state=42, stratify=y
)

# 6. ESCALONAMENTO
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ==============================================================================
# 7. IMPLEMENTAÇÃO E AVALIAÇÃO DO MODELO NAIVE BAYES (H4)
# ==============================================================================
modelo_nb = GaussianNB()
modelo_nb.fit(X_train_scaled, y_train)
y_pred = modelo_nb.predict(X_test_scaled)

# 8. Avaliação de Desempenho
acuracia = accuracy_score(y_test, y_pred)
relatorio = classification_report(y_test, y_pred, target_names=classes_finais, zero_division=0)

print("\n=== H4: CLASSIFICAÇÃO COM NAIVE BAYES (SOLUÇÃO FINAL) ===")
print(f"Acurácia Geral: {acuracia:.4f}")
print("\n--- Relatório de Desempenho (Por Gênero) ---")
print(relatorio)

# 9. Matriz de Confusão para Visualização
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=classes_finais, yticklabels=classes_finais)
plt.title('Matriz de Confusão - Naive Bayes (Classificação de Gêneros)', fontsize=16)
plt.xlabel('Gênero Previsto', fontsize=12)
plt.ylabel('Gênero Real', fontsize=12)
plt.show()

###### Acurácia Geral que obtivemos: 0.4206 (42.06%)

**Conclusão Final (H4)**: As Características de Áudio Definem Gêneros

 Veredito: Hipótese Validada (Com Limitações)
A aplicação do modelo Gaussian Naive Bayes para classificar músicas em 10 gêneros principais resultou em uma Acurácia Geral de 42.06%.

- **Validação da H4**: Este resultado é mais de quatro vezes superior à chance de acerto aleatório (que seria de apenas 10%, dado que há 10 classes). Isso prova que as características de áudio (energia, acustica, bpm, etc.) são, de fato, capazes de criar uma assinatura digital que o modelo pode reconhecer.

- **Limitação**: A acurácia não é perfeita, pois muitas características se sobrepõem na música moderna (ex: um pop eletrônico tem características de "dance" e "pop"), o que leva à confusão entre gêneros.

#  Hipótese H5: Regressão Logística Binária (Pop vs Rock)

## Pergunta Central:

Dado que a Classificação Multiclasse (H4) demonstrou confusão entre gêneros próximos (como Pop e Rock), é possível que o modelo de **Regressão Logística** consiga isolar as diferenças cruciais para distinguir apenas essas duas classes com alta precisão?

## Objetivo do Modelo:

O modelo de Regressão Logística será aplicado de forma **Binária** (Pop = 1; Rock = 0) para testar o seguinte:

1.  **Acurácia:** Conseguiremos atingir uma Acurácia acima de **80%** na distinção entre Pop e Rock?
2.  **Interpretação (Coeficientes):** Qual variável é o maior preditor de Pop (coeficiente positivo) e qual é o maior preditor de Rock (coeficiente negativo)?

## Variáveis Envolvidas:

| Tipo | Variável | Função Esperada |
| :--- | :--- | :--- |
| **Target (y)** | `genero` (Pop ou Rock) | Pop (1) e Rock (0). |
| **Features (X)** | `energia`, `volume` | Espera-se que sejam **Positivos** (Favorecem o Pop). |
| **Features (X)** | `acustica` | Espera-se que seja **Negativo** (Favorece o Rock). |
| **Features (X)** | `dancabilidade`, `valencia` | Fatores que podem ser positivos, mas menos polares. |

---

*A Regressão Logística fornecerá os **coeficientes** padronizados, que nos permitirão traçar o perfil exato de produção que separa as baladas de Rock dos hits de Pop.*

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# 1. CARREGAMENTO E LIMPEZA DE DADOS (Robusto)
df = pd.read_csv('/content/RegressaoLinear/data/processed/dataset_limpo.csv')
df_clean = df[
    (df['duracao_ms'] >= 30000) & (df['duracao_ms'] <= 600000) &
    (df['tempo_bpm'] > 0) & (df['tempo_bpm'] <= 250) &
    (df['volume'] >= -35)
].copy()

# 2. FILTRAGEM BINÁRIA (Pop vs Rock)
# Criamos o novo dataset apenas com os dois gêneros de interesse
df_binary = df_clean[df_clean['genero'].isin(['pop', 'rock'])].copy()

# 3. Definição de X (Features) e y (Target Binário)
features = ['dancabilidade', 'energia', 'volume', 'acustica', 'valencia', 'tempo_bpm']
X = df_binary[features].values
# Codificação: Pop = 1, Rock = 0
y = (df_binary['genero'] == 'pop').astype(int).values
classes_finais = ['Rock (0)', 'Pop (1)']

# 4. Divisão Treino/Teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. ESCALONAMENTO (Essencial para Regressão Logística)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ==============================================================================
# 6. IMPLEMENTAÇÃO E AVALIAÇÃO DO MODELO DE REGRESSÃO LOGÍSTICA
# ==============================================================================
# C=1.0 é a penalidade padrão (Regularização L2)
modelo_logistico = LogisticRegression(solver='liblinear', C=1.0, random_state=42)
modelo_logistico.fit(X_train_scaled, y_train)
y_pred = modelo_logistico.predict(X_test_scaled)

# 7. Avaliação de Desempenho
acuracia = accuracy_score(y_test, y_pred)
relatorio = classification_report(y_test, y_pred, target_names=classes_finais, zero_division=0)

# 8. Extração dos Coeficientes (O mais importante!)
coeficientes = pd.DataFrame({
    'Feature': features,
    'Coeficiente': modelo_logistico.coef_[0].round(4)
}).sort_values(by='Coeficiente', ascending=False)

print("\n=== H5: REGRESSÃO LOGÍSTICA (Pop vs Rock) ===")
print(f"Acurácia: {acuracia:.4f}")
print("\n--- Relatório de Classificação ---")
print(relatorio)

print("\n--- Coeficientes do Modelo (Importância) ---")
print("Coeficiente Positivo = Aumenta a chance de ser POP (1)")
print("Coeficiente Negativo = Aumenta a chance de ser ROCK (0)")
print(coeficientes.to_markdown(index=False))

# 🎸 Conclusão da Hipótese H5: Regressão Logística (Pop vs Rock)

### Veredito: Classificação Binária com Alta Precisão e Surpresas ✅

A **Regressão Logística** foi utilizada para classificar de forma binária os gêneros **Pop** (Target=1) e **Rock** (Target=0). O modelo demonstrou alto poder de distinção, mas com um perfil de características que surpreende as expectativas iniciais.



**1. Desempenho do Modelo**
* **Acurácia:** O modelo atingiu uma Acurácia de **73.25%** (0.7325).
* **Interpretação:** Uma acurácia de **73%** é excelente para uma classificação de dois gêneros que são frequentemente confundidos por modelos simples. Isso prova que a Regressão Logística conseguiu traçar uma **fronteira de decisão** clara entre o perfil acústico do Pop e do Rock.
* **Relatório:** A Precisão (0.74) e o Recall (0.76) são equilibrados, indicando que o modelo acerta a previsão para ambos os gêneros de forma consistente.

**2. Análise dos Coeficientes (O Fator Determinante)**

Os coeficientes padronizados são a chave, pois mostram a direção e a força com que cada *feature* influencia a decisão do modelo.

| Ranking | Feature | Coeficiente | Classificação | Interpretação |
| :--- | :--- | :--- | :--- | :--- |
| **1º** | **dancabilidade** | **+1.0109** | **Pop** | **Preditor mais Forte:** Músicas dançantes são o maior indicador de Pop. |
| **2º** | **acustica** | **+0.6668** | **Pop** | **A Maior Surpresa:** Alto índice acústico indica **Pop** (contrariando a intuição de que Rock é mais orgânico). |
| **3º** | **volume** | **+0.3627** | **Pop** | Músicas mais altas (Volume) favorecem o Pop. |
| **4º** | **valencia** | **-0.3880** | **Rock** | **Preditor mais Forte do Rock:** Músicas menos "felizes" (Valência mais baixa) indicam Rock. |
| **5º** | **energia** | **-0.2112** | **Rock** | Menos Energia favorece o Rock (ao contrário da H1). |

**Conclusão Final (H5):**

O modelo revela que o perfil do **Pop** é definido por **Dancabilidade** e, surpreendentemente, por uma alta pontuação de **Acústica** (o que pode refletir o Pop moderno, *desplugado* ou com produção limpa, sendo menos distorcido que o Rock).

O **Rock**, por sua vez, é caracterizado por ser **Menos Feliz** (`valencia` negativa) e ter um pouco menos de `Energia`, provando que o perfil desses dois gêneros se diferencia de formas inesperadas.

In [3]:
import pandas as pd
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# --- Replicando a preparação de dados (necessária para rodar o modelo) ---
df = pd.read_csv('/content/RegressaoLinear/data/processed/dataset_limpo.csv')
df_clean = df[
    (df['duracao_ms'] >= 30000) & (df['duracao_ms'] <= 600000) &
    (df['tempo_bpm'] > 0) & (df['tempo_bpm'] <= 250) &
    (df['volume'] >= -35)
].copy()
features = ['dancabilidade', 'energia', 'volume', 'falada',
            'acustica', 'instrumental', 'ao_vivo', 'valencia', 'tempo_bpm',
            'duracao_ms']
target = 'genero'
df_clf = df_clean[features + [target]].copy()
generos_validos = df_clf['genero'].value_counts()
generos_validos = generos_validos[generos_validos >= 1000].index
df_clf = df_clf[df_clf['genero'].isin(generos_validos)].reset_index(drop=True)
le = LabelEncoder()
y = le.fit_transform(df_clf[target])
X = df_clf[features].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# --- Fim da preparação ---

# Configurando o modelo e o K-Fold
modelo_nb = GaussianNB()
# StratifiedKFold garante que a proporção dos gêneros seja mantida em cada fold
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Aplicando a Validação Cruzada (usaremos 'accuracy' como métrica)
scores = cross_val_score(modelo_nb, X_scaled, y, cv=cv, scoring='accuracy')

print("=== Validação Cruzada (Cross-Validation) - Naive Bayes ===")
print(f"Scores por Fold: {scores.round(4)}")
print(f"Acurácia Média (5 Folds): {scores.mean():.4f}")
print(f"Desvio Padrão: {scores.std():.4f}")

=== Validação Cruzada (Cross-Validation) - Naive Bayes ===
Scores por Fold: [0.1835 0.1693 0.1798 0.1714 0.1779]
Acurácia Média (5 Folds): 0.1764
Desvio Padrão: 0.0053


In [7]:
import time
from sklearn.ensemble import RandomForestClassifier
# Trocamos GridSearchCV por RandomizedSearchCV!
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np

# --- Replicando a preparação de dados ---
# X_scaled e y devem estar carregados na memória do seu notebook.
df = pd.read_csv('/content/RegressaoLinear/data/processed/dataset_limpo.csv')
df_clean = df[
    (df['duracao_ms'] >= 30000) & (df['duracao_ms'] <= 600000) &
    (df['tempo_bpm'] > 0) & (df['tempo_bpm'] <= 250) &
    (df['volume'] >= -35)
].copy()
features = ['dancabilidade', 'energia', 'volume', 'falada',
            'acustica', 'instrumental', 'ao_vivo', 'valencia', 'tempo_bpm',
            'duracao_ms']
target = 'genero'
df_clf = df_clean[features + [target]].copy()
generos_validos = df_clf['genero'].value_counts()
generos_validos = generos_validos[generos_validos >= 1000].index
df_clf = df_clf[df_clf['genero'].isin(generos_validos)].reset_index(drop=True)
le = LabelEncoder()
y = le.fit_transform(df_clf[target])
X = df_clf[features].values
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# --- Fim da preparação ---

# Definindo o espaço de busca (Distribuição)
# Usamos distribuições estatísticas (randint) no lugar de listas fixas
param_dist = {
    'n_estimators': randint(50, 250),      # Nº de árvores entre 50 e 250
    'max_depth': randint(10, 40),          # Profundidade entre 10 e 40
    'min_samples_leaf': randint(1, 10)     # Folhas entre 1 e 10
}

# Configurando o Random Search
random_search = RandomizedSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_distributions=param_dist,
    n_iter=10, # Testar apenas 10 combinações aleatórias
    scoring='accuracy',
    cv=3,
    n_jobs=-1,
    random_state=42,
    verbose=1
)

print("\n=== Otimização RÁPIDA: Random Search (Random Forest) ===")
start_time = time.time()

# Rodando o Random Search
random_search.fit(X_scaled, y)

end_time = time.time()
tempo_total = end_time - start_time

print(f"Tempo total de execução do Random Search: {tempo_total:.2f} segundos")
print(f"Melhor Acurácia Encontrada (Média CV): {random_search.best_score_:.4f}")
print(f"Melhores Parâmetros: {random_search.best_params_}")

# Registrando os resultados finais
best_rf_model = random_search.best_estimator_
final_acuracia_treino = best_rf_model.score(X_scaled, y)
print(f"Acurácia Final do Melhor Modelo no Dataset Completo: {final_acuracia_treino:.4f}")


=== Otimização RÁPIDA: Random Search (Random Forest) ===
Fitting 3 folds for each of 10 candidates, totalling 30 fits


/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Tempo total de execução do Random Search: 623.70 segundos
Melhor Acurácia Encontrada (Média CV): 0.2835
Melhores Parâmetros: {'max_depth': 37, 'min_samples_leaf': 5, 'n_estimators': 210}
Acurácia Final do Melhor Modelo no Dataset Completo: 0.7339
